In [28]:
import numpy as np
import random

# Grid dimensions
GRID_SIZE = 100

grid = np.zeros((GRID_SIZE, GRID_SIZE))

# Place random obstacles
num_obstacles = int(GRID_SIZE * GRID_SIZE * 0.2)  # 20% of the grid as obstacles
for _ in range(num_obstacles):
    x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
    grid[x, y] = -1  # -1 indicates an obstacle

# Define start and goal points
start = (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1))
goal = (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1))

# Ensure start and goal are not obstacles
grid[start] = 0
grid[goal] = 0


In [29]:
# Actions (Up, Down, Left, Right)
ACTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Step function to handle boundaries and obstacles
def step(state, action):
    x, y = state
    dx, dy = action
    nx, ny = x + dx, y + dy
    
    # Ensure next state is within bounds
    if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE and grid[nx, ny] != -1:
        return (nx, ny), -1  # Move with a cost
    return state, -1  # No movement if obstacle or out of bounds

In [5]:
def value_iteration(theta=0.0001, discount_factor=0.9):
    V = np.zeros((GRID_SIZE, GRID_SIZE))
    policy = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))
    
    while True:
        delta = 0
        for x in range(GRID_SIZE):
            for y in range(GRID_SIZE):
                if (x, y) == goal or grid[x, y] == -1:
                    continue
                
                v = V[x, y]
                q_values = []
                
                for a_idx, action in enumerate(ACTIONS):
                    next_state, reward = step((x, y), action)
                    next_x, next_y = next_state
                    q_value = reward + discount_factor * V[next_x, next_y]
                    q_values.append(q_value)
                
                V[x, y] = max(q_values)
                policy[x, y] = np.eye(len(ACTIONS))[np.argmax(q_values)]
                
                delta = max(delta, abs(v - V[x, y]))
        
        if delta < theta:
            break
    return policy, V


In [31]:
# Q-Learning Algorithm
def q_learning(episodes=500, alpha=0.1, gamma=0.9, epsilon=0.3):
    Q = defaultdict(lambda: np.zeros(len(ACTIONS)))
    
    for _ in range(episodes):
        state = start
        while state != goal:
            if random.uniform(0, 1) < epsilon:
                action_index = random.choice(range(len(ACTIONS)))
            else:
                action_index = np.argmax(Q[state])
            
            action = ACTIONS[action_index]
            next_state, reward = step(state, action)
            
            best_next_action = np.argmax(Q[next_state])
            Q[state][action_index] += alpha * (reward + gamma * Q[next_state][best_next_action] - Q[state][action_index])
            
            state = next_state
    return Q

In [30]:
# SARSA Algorithm
def sarsa(episodes=500, alpha=0.1, gamma=0.9, epsilon=0.3):
    Q = defaultdict(lambda: np.zeros(len(ACTIONS)))
    
    for _ in range(episodes):
        state = start
        if random.uniform(0, 1) < epsilon:
            action_index = random.choice(range(len(ACTIONS)))
        else:
            action_index = np.argmax(Q[state])
        
        while state != goal:
            action = ACTIONS[action_index]
            next_state, reward = step(state, action)
            
            if random.uniform(0, 1) < epsilon:
                next_action_index = random.choice(range(len(ACTIONS)))
            else:
                next_action_index = np.argmax(Q[next_state])
                
            Q[state][action_index] += alpha * (reward + gamma * Q[next_state][next_action_index] - Q[state][action_index])
            
            state = next_state
            action_index = next_action_index
    return Q

In [32]:
policy_dp, V_dp = value_iteration()
Q_sarsa = sarsa()
Q_qlearning = q_learning()

print("DP Value Function:", V_dp)
print("Q-Learning Q-Values:", Q_qlearning)
print("SARSA Q-Values:", Q_sarsa)


DP Value Function: [[-9.99757251  0.         -9.99700309 ... -9.97261073 -9.97534965
  -9.97781469]
 [-9.99730278 -9.99700309 -9.9966701  ... -9.96956747 -9.97261073
   0.        ]
 [-9.99700309  0.         -9.99630012 ... -9.96618608  0.
  -9.97261073]
 ...
 [-9.99915359 -9.99915359  0.         ... -9.99915359 -9.99915359
   0.        ]
 [ 0.         -9.99915359 -9.99915359 ... -9.99915359 -9.99915359
  -9.99915359]
 [-9.99915359 -9.99915359 -9.99915359 ... -9.99915359 -9.99915359
  -9.99915359]]
Q-Learning Q-Values: defaultdict(<function q_learning.<locals>.<lambda> at 0x0000019D2367E8E0>, {(76, 19): array([-8.33533703, -8.34009968, -8.35525357, -8.33299371]), (76, 18): array([-8.421178  , -8.4258387 , -8.41929083, -8.42557907]), (76, 20): array([-8.24832438, -8.24877578, -8.26848585, -8.2381858 ]), (75, 20): array([-8.23499141, -8.22432198, -8.22761055, -8.22695086]), (74, 20): array([-8.17854318, -8.20563879, -8.18689433, -8.18943616]), (73, 20): array([-8.13536397, -8.15267984, -8

In [33]:
# Convert one-hot encoded policy_dp to action indices
policy_dp_indices = np.argmax(policy_dp, axis=2)



In [34]:
import numpy as np

def evaluate_policy(Q, episodes=100, max_steps=1000):
    steps_to_goal = []
    cumulative_rewards = []
    success_count = 0

    for _ in range(episodes):
        state = start  
        total_reward = 0
        steps = 0

        while state != goal and steps < max_steps:
            if isinstance(Q, dict):  #  Q-learning or SARSA
                action_index = np.argmax(Q[state])  #  best action from Q-values
            else:  #  DP policy, assume Q is a policy array
                action_index = Q[state]
            action = ACTIONS[action_index]
            next_state, reward = step(state, action)
            
            total_reward += reward
            steps += 1
            state = next_state

        # Track results
        cumulative_rewards.append(total_reward)
        steps_to_goal.append(steps)
        if state == goal:
            success_count += 1

    avg_steps = sum(steps_to_goal) / episodes
    avg_reward = sum(cumulative_rewards) / episodes
    success_rate = (success_count / episodes) * 100

    return {
        "Average Steps to Goal": avg_steps,
        "Average Cumulative Reward": avg_reward,
        "Success Rate (%)": success_rate
    }


In [21]:
dp_benchmark = evaluate_policy(policy_dp_indices)
q_learning_benchmark = evaluate_policy(Q_qlearning)
sarsa_benchmark = evaluate_policy(Q_sarsa)


print("Benchmark Results")
print("DP Benchmark:", dp_benchmark)
print("Q-Learning Benchmark:", q_learning_benchmark)
print("SARSA Benchmark:", sarsa_benchmark)

Benchmark Results
DP Benchmark: {'Average Steps to Goal': 85.0, 'Average Cumulative Reward': 16.0, 'Success Rate (%)': 100.0}
Q-Learning Benchmark: {'Average Steps to Goal': 1000.0, 'Average Cumulative Reward': -1000.0, 'Success Rate (%)': 0.0}
SARSA Benchmark: {'Average Steps to Goal': 1000.0, 'Average Cumulative Reward': -1000.0, 'Success Rate (%)': 0.0}
